<a href="https://colab.research.google.com/github/minmincg/house_pricing_analysis/blob/main/Notebooks/neural_networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#Import the csv by using the raw link
hpricing_df_og = pd.read_csv("https://raw.githubusercontent.com/minmincg/house_pricing_analysis/24b41d230570324f8d64a1326bcc9e9972a62608/Resources/austin_housing_reduced.csv")
hpricing_df_og.head()

,city,streetAddress,zipcode,latitude,longitude,propertyTaxRate,garageSpaces,hasCooling,hasGarage,hasHeating,...,numOfWaterfrontFeatures,numOfWindowFeatures,numOfCommunityFeatures,lotSizeSqFt,livingAreaSqFt,avgSchoolRating,numOfBathrooms,numOfBedrooms,numOfStories,numOfSchools
0,pflugerville,14424 Lake Victor Dr,78660,30.430632,-97.663078,1.98,2,True,True,True,...,0,1,0,6011.0,2601,2.666667,3.0,4,2,3
1,pflugerville,1104 Strickling Dr,78660,30.432672,-97.661697,1.98,2,True,True,True,...,0,0,0,6185.0,1768,2.666667,2.0,4,1,3
2,pflugerville,1408 Fort Dessau Rd,78660,30.409748,-97.639771,1.98,0,True,False,True,...,0,0,0,7840.0,1478,3.000000,2.0,3,1,4
3,pflugerville,1025 Strickling Dr,78660,30.432112,-97.661659,1.98,2,True,True,True,...,0,0,0,6098.0,1678,2.666667,2.0,3,1,3
4,pflugerville,15005 Donna Jane Loop,78660,30.437368,-97.656860,1.98,0,True,False,True,...,0,0,0,6708.0,2132,4.000000,3.0,3,2,3


In [2]:
# Determine the number of unique values in each column.
hpricing_df_og.nunique()

city                              9
streetAddress                 15000
zipcode                          48
latitude                      14269
longitude                     12379
propertyTaxRate                   3
garageSpaces                     14
hasCooling                        2
hasGarage                         2
hasHeating                        2
hasSpa                            2
hasView                           2
homeType                         10
yearBuilt                       115
latestPrice                    2338
numPriceChanges                  22
numOfAccessibilityFeatures        8
numOfAppliances                  13
numOfParkingFeatures              7
numOfPatioAndPorchFeatures        9
numOfSecurityFeatures             7
numOfWaterfrontFeatures           3
numOfWindowFeatures               5
numOfCommunityFeatures            9
lotSizeSqFt                    1403
livingAreaSqFt                 3616
avgSchoolRating                  27
numOfBathrooms              

In [3]:
# Drop the non-beneficial ID columns, 'city' and 'streetAddress'.
hpricing_df = hpricing_df_og.drop(columns = ['city', 
                                             'streetAddress',
                                             'latitude',
                                             'longitude',
#                                              'numPriceChanges',
                                             'homeType',
#                                              'zipcode',
                                             'avgSchoolRating',
                                             'numOfSchools'
                                            ], axis=1)
hpricing_df.head()

,zipcode,propertyTaxRate,garageSpaces,hasCooling,hasGarage,hasHeating,hasSpa,hasView,yearBuilt,latestPrice,...,numOfPatioAndPorchFeatures,numOfSecurityFeatures,numOfWaterfrontFeatures,numOfWindowFeatures,numOfCommunityFeatures,lotSizeSqFt,livingAreaSqFt,numOfBathrooms,numOfBedrooms,numOfStories
0,78660,1.98,2,True,True,True,False,False,2012,305000,...,1,3,0,1,0,6011.0,2601,3.0,4,2
1,78660,1.98,2,True,True,True,False,False,2013,295000,...,0,0,0,0,0,6185.0,1768,2.0,4,1
2,78660,1.98,0,True,False,True,False,False,2018,256125,...,0,1,0,0,0,7840.0,1478,2.0,3,1
3,78660,1.98,2,True,True,True,False,False,2013,240000,...,0,0,0,0,0,6098.0,1678,2.0,3,1
4,78660,1.98,0,True,False,True,False,False,2002,239900,...,0,0,0,0,0,6708.0,2132,3.0,3,2


In [4]:
# Making sure the columns got dropped
hpricing_df.columns

Index(['zipcode', 'propertyTaxRate', 'garageSpaces', 'hasCooling', 'hasGarage',
       'hasHeating', 'hasSpa', 'hasView', 'yearBuilt', 'latestPrice',
       'numPriceChanges', 'numOfAccessibilityFeatures', 'numOfAppliances',
       'numOfParkingFeatures', 'numOfPatioAndPorchFeatures',
       'numOfSecurityFeatures', 'numOfWaterfrontFeatures',
       'numOfWindowFeatures', 'numOfCommunityFeatures', 'lotSizeSqFt',
       'livingAreaSqFt', 'numOfBathrooms', 'numOfBedrooms', 'numOfStories'],
      dtype='object')

In [5]:
# Look at 'zipcode' value counts for binning
zip_counts = hpricing_df["zipcode"].value_counts()
zip_counts

78748    1137
78745    1025
78749     772
78704     693
78737     611
78739     603
78732     590
78759     559
78723     554
78757     518
78717     515
78731     487
78744     469
78753     438
78727     414
78754     393
78758     393
78702     382
78747     381
78750     321
78729     317
78735     308
78703     294
78721     244
78730     243
78728     224
78724     223
78741     217
78733     202
78746     190
78726     178
78751     172
78725     162
78736     162
78752     151
78756     137
78722      96
78617      85
78705      64
78660      36
78619      12
78734       9
78701       8
78719       5
78652       4
78653       4
78742       4
78738       1
Name: zipcode, dtype: int64

In [6]:
# Choose a cutoff value of less than 200 and create a list of application types to be replaced

zipcodes_to_replace = list(zip_counts[zip_counts<200].index)

# Replace in dataframe
for zip in zipcodes_to_replace:
    hpricing_df['zipcode'] = hpricing_df['zipcode'].replace(zip,"Other")

# Making sure the binning was successful
hpricing_df['zipcode'].value_counts()

Other    1480
78748    1137
78745    1025
78749     772
78704     693
78737     611
78739     603
78732     590
78759     559
78723     554
78757     518
78717     515
78731     487
78744     469
78753     438
78727     414
78758     393
78754     393
78702     382
78747     381
78750     321
78729     317
78735     308
78703     294
78721     244
78730     243
78728     224
78724     223
78741     217
78733     202
Name: zipcode, dtype: int64

In [7]:
# hometype_counts=hpricing_df['homeType'].value_counts()

# hometype_to_replace = list(hometype_counts[hometype_counts<50].index)

# # Replace in dataframe
# for hometype in hometype_to_replace:
#     hpricing_df['homeType'] = hpricing_df['homeType'].replace(hometype,"Other")

# # Making sure the binning was successful
# hpricing_df['homeType'].value_counts()

In [8]:
#Check data types
hpricing_df.dtypes

zipcode                        object
propertyTaxRate               float64
garageSpaces                    int64
hasCooling                       bool
hasGarage                        bool
hasHeating                       bool
hasSpa                           bool
hasView                          bool
yearBuilt                       int64
latestPrice                     int64
numPriceChanges                 int64
numOfAccessibilityFeatures      int64
numOfAppliances                 int64
numOfParkingFeatures            int64
numOfPatioAndPorchFeatures      int64
numOfSecurityFeatures           int64
numOfWaterfrontFeatures         int64
numOfWindowFeatures             int64
numOfCommunityFeatures          int64
lotSizeSqFt                   float64
livingAreaSqFt                  int64
numOfBathrooms                float64
numOfBedrooms                   int64
numOfStories                    int64
dtype: object

In [9]:
#Convert zipcode to dummies because it is a categorical
#Didnt use dummies in the rest of the data (hasCooling, hasGarage, hasHeating, hasSpa, hasView, homeType) because it was already numerical
hpricing_df = pd.get_dummies(hpricing_df,dtype=float)
hpricing_df

#hpricing_df['zipcode']= hpricing_df['zipcode'].astype(float)
#hpricing_df.dtypes

,propertyTaxRate,garageSpaces,hasCooling,hasGarage,hasHeating,hasSpa,hasView,yearBuilt,latestPrice,numPriceChanges,...,zipcode_78747,zipcode_78748,zipcode_78749,zipcode_78750,zipcode_78753,zipcode_78754,zipcode_78757,zipcode_78758,zipcode_78759,zipcode_Other
0,1.98,2,True,True,True,False,False,2012,305000,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,1.98,2,True,True,True,False,False,2013,295000,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,1.98,0,True,False,True,False,False,2018,256125,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,1.98,2,True,True,True,False,False,2013,240000,4,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,1.98,0,True,False,True,False,False,2002,239900,3,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15002,1.98,6,True,True,True,False,True,1986,330000,1,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15003,1.98,2,True,True,True,True,True,2016,550000,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15004,1.98,2,True,True,True,False,False,1936,875000,7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15005,1.98,0,True,False,True,False,False,1968,420000,5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [10]:
hpricing_df.columns

Index(['propertyTaxRate', 'garageSpaces', 'hasCooling', 'hasGarage',
       'hasHeating', 'hasSpa', 'hasView', 'yearBuilt', 'latestPrice',
       'numPriceChanges', 'numOfAccessibilityFeatures', 'numOfAppliances',
       'numOfParkingFeatures', 'numOfPatioAndPorchFeatures',
       'numOfSecurityFeatures', 'numOfWaterfrontFeatures',
       'numOfWindowFeatures', 'numOfCommunityFeatures', 'lotSizeSqFt',
       'livingAreaSqFt', 'numOfBathrooms', 'numOfBedrooms', 'numOfStories',
       'zipcode_78702', 'zipcode_78703', 'zipcode_78704', 'zipcode_78717',
       'zipcode_78721', 'zipcode_78723', 'zipcode_78724', 'zipcode_78727',
       'zipcode_78728', 'zipcode_78729', 'zipcode_78730', 'zipcode_78731',
       'zipcode_78732', 'zipcode_78733', 'zipcode_78735', 'zipcode_78737',
       'zipcode_78739', 'zipcode_78741', 'zipcode_78744', 'zipcode_78745',
       'zipcode_78747', 'zipcode_78748', 'zipcode_78749', 'zipcode_78750',
       'zipcode_78753', 'zipcode_78754', 'zipcode_78757', 'zipcode_78

In [11]:
# Split our preprocessed data into our features and target arrays
y= hpricing_df['latestPrice'].values

X= hpricing_df.drop ('latestPrice', axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)

In [12]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [27]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="relu"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 80)                4240      
                                                                 
 dense_4 (Dense)             (None, 30)                2430      
                                                                 
 dense_5 (Dense)             (None, 1)                 31        
                                                                 
Total params: 6,701
Trainable params: 6,701
Non-trainable params: 0
_________________________________________________________________


In [28]:
# Compile the model
#mae = mean absolute error & mse = mean squared error
nn.compile(loss="mse", optimizer="adam", metrics=["mae", "mse"])

In [29]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=500)

Epoch 1/500
352/352 [==============================] - 1s 1ms/step - loss: 418608283648.0000 - mae: 508200.8750 - mse: 418608283648.0000
Epoch 2/500
352/352 [==============================] - 0s 1ms/step - loss: 412497313792.0000 - mae: 504359.6562 - mse: 412497313792.0000
Epoch 3/500
352/352 [==============================] - 0s 1ms/step - loss: 390087639040.0000 - mae: 490057.0000 - mse: 390087639040.0000
Epoch 4/500
352/352 [==============================] - 0s 1ms/step - loss: 346768179200.0000 - mae: 461013.8125 - mse: 346768179200.0000
Epoch 5/500
352/352 [==============================] - 0s 1ms/step - loss: 290596487168.0000 - mae: 417633.0000 - mse: 290596487168.0000
Epoch 6/500
352/352 [==============================] - 0s 1ms/step - loss: 235591876608.0000 - mae: 369298.3750 - mse: 235591876608.0000
Epoch 7/500
352/352 [==============================] - 0s 1ms/step - loss: 192553058304.0000 - mae: 331111.8438 - mse: 192553058304.0000
Epoch 8/500
352/352 [====================

352/352 [==============================] - 0s 1ms/step - loss: 50465914880.0000 - mae: 117474.4766 - mse: 50465914880.0000
Epoch 62/500
352/352 [==============================] - 0s 1ms/step - loss: 50349875200.0000 - mae: 116911.6875 - mse: 50349875200.0000
Epoch 63/500
352/352 [==============================] - 0s 1ms/step - loss: 50236645376.0000 - mae: 117046.9609 - mse: 50236645376.0000
Epoch 64/500
352/352 [==============================] - 0s 1ms/step - loss: 50120663040.0000 - mae: 116686.5859 - mse: 50120663040.0000
Epoch 65/500
352/352 [==============================] - 0s 1ms/step - loss: 50025271296.0000 - mae: 116730.7266 - mse: 50025271296.0000
Epoch 66/500
352/352 [==============================] - 0s 1ms/step - loss: 49933205504.0000 - mae: 116653.1641 - mse: 49933205504.0000
Epoch 67/500
352/352 [==============================] - 0s 1ms/step - loss: 49834283008.0000 - mae: 116316.3438 - mse: 49834283008.0000
Epoch 68/500
352/352 [==============================] - 0s 1m

352/352 [==============================] - 0s 1ms/step - loss: 46594699264.0000 - mae: 110905.9922 - mse: 46594699264.0000
Epoch 122/500
352/352 [==============================] - 0s 1ms/step - loss: 46551797760.0000 - mae: 110837.1172 - mse: 46551797760.0000
Epoch 123/500
352/352 [==============================] - 0s 1ms/step - loss: 46502256640.0000 - mae: 110869.9844 - mse: 46502256640.0000
Epoch 124/500
352/352 [==============================] - 0s 1ms/step - loss: 46469263360.0000 - mae: 110786.5156 - mse: 46469263360.0000
Epoch 125/500
352/352 [==============================] - 0s 1ms/step - loss: 46417309696.0000 - mae: 110705.5078 - mse: 46417309696.0000
Epoch 126/500
352/352 [==============================] - 0s 1ms/step - loss: 46379831296.0000 - mae: 110468.7109 - mse: 46379831296.0000
Epoch 127/500
352/352 [==============================] - 0s 1ms/step - loss: 46339952640.0000 - mae: 110457.9141 - mse: 46339952640.0000
Epoch 128/500
352/352 [==============================] 

352/352 [==============================] - 0s 1ms/step - loss: 44710207488.0000 - mae: 106907.6562 - mse: 44710207488.0000
Epoch 181/500
352/352 [==============================] - 0s 1ms/step - loss: 44674461696.0000 - mae: 106885.5234 - mse: 44674461696.0000
Epoch 182/500
352/352 [==============================] - 0s 1ms/step - loss: 44655312896.0000 - mae: 106975.0781 - mse: 44655312896.0000
Epoch 183/500
352/352 [==============================] - 0s 1ms/step - loss: 44625199104.0000 - mae: 106903.4297 - mse: 44625199104.0000
Epoch 184/500
352/352 [==============================] - 0s 1ms/step - loss: 44595838976.0000 - mae: 106806.1797 - mse: 44595838976.0000
Epoch 185/500
352/352 [==============================] - 1s 1ms/step - loss: 44586471424.0000 - mae: 106774.2812 - mse: 44586471424.0000
Epoch 186/500
352/352 [==============================] - 1s 1ms/step - loss: 44550365184.0000 - mae: 106619.6797 - mse: 44550365184.0000
Epoch 187/500
352/352 [==============================] 

352/352 [==============================] - 0s 1ms/step - loss: 43512967168.0000 - mae: 104573.4766 - mse: 43512967168.0000
Epoch 240/500
352/352 [==============================] - 0s 1ms/step - loss: 43486965760.0000 - mae: 104773.0781 - mse: 43486965760.0000
Epoch 241/500
352/352 [==============================] - 0s 1ms/step - loss: 43481464832.0000 - mae: 104293.8906 - mse: 43481464832.0000
Epoch 242/500
352/352 [==============================] - 0s 1ms/step - loss: 43467935744.0000 - mae: 104268.2344 - mse: 43467935744.0000
Epoch 243/500
352/352 [==============================] - 0s 1ms/step - loss: 43438903296.0000 - mae: 104497.4141 - mse: 43438903296.0000
Epoch 244/500
352/352 [==============================] - 0s 1ms/step - loss: 43440881664.0000 - mae: 104187.2500 - mse: 43440881664.0000
Epoch 245/500
352/352 [==============================] - 0s 1ms/step - loss: 43410821120.0000 - mae: 104191.0703 - mse: 43410821120.0000
Epoch 246/500
352/352 [==============================] 

352/352 [==============================] - 0s 1ms/step - loss: 42727690240.0000 - mae: 102910.4062 - mse: 42727690240.0000
Epoch 299/500
352/352 [==============================] - 0s 1ms/step - loss: 42694795264.0000 - mae: 102810.5547 - mse: 42694795264.0000
Epoch 300/500
352/352 [==============================] - 0s 1ms/step - loss: 42697261056.0000 - mae: 103157.1484 - mse: 42697261056.0000
Epoch 301/500
352/352 [==============================] - 0s 1ms/step - loss: 42682437632.0000 - mae: 102601.1484 - mse: 42682437632.0000
Epoch 302/500
352/352 [==============================] - 0s 1ms/step - loss: 42701049856.0000 - mae: 103107.9297 - mse: 42701049856.0000
Epoch 303/500
352/352 [==============================] - 0s 1ms/step - loss: 42672926720.0000 - mae: 102860.4844 - mse: 42672926720.0000
Epoch 304/500
352/352 [==============================] - 0s 1ms/step - loss: 42697105408.0000 - mae: 102950.4609 - mse: 42697105408.0000
Epoch 305/500
352/352 [==============================] 

352/352 [==============================] - 1s 3ms/step - loss: 42102857728.0000 - mae: 101848.3203 - mse: 42102857728.0000
Epoch 358/500
352/352 [==============================] - 1s 3ms/step - loss: 42087354368.0000 - mae: 101789.3203 - mse: 42087354368.0000
Epoch 359/500
352/352 [==============================] - 1s 3ms/step - loss: 42076123136.0000 - mae: 101756.5859 - mse: 42076123136.0000
Epoch 360/500
352/352 [==============================] - 1s 3ms/step - loss: 42082078720.0000 - mae: 101669.8906 - mse: 42082078720.0000
Epoch 361/500
352/352 [==============================] - 1s 3ms/step - loss: 42063958016.0000 - mae: 101935.2422 - mse: 42063958016.0000
Epoch 362/500
352/352 [==============================] - 1s 3ms/step - loss: 42057981952.0000 - mae: 101586.4062 - mse: 42057981952.0000
Epoch 363/500
352/352 [==============================] - 1s 3ms/step - loss: 42036224000.0000 - mae: 101659.9766 - mse: 42036224000.0000
Epoch 364/500
352/352 [==============================] 

352/352 [==============================] - 1s 3ms/step - loss: 41568735232.0000 - mae: 100465.6953 - mse: 41568735232.0000
Epoch 417/500
352/352 [==============================] - 1s 3ms/step - loss: 41534320640.0000 - mae: 100594.6797 - mse: 41534320640.0000
Epoch 418/500
352/352 [==============================] - 1s 3ms/step - loss: 41542656000.0000 - mae: 100707.2734 - mse: 41542656000.0000
Epoch 419/500
352/352 [==============================] - 1s 3ms/step - loss: 41557086208.0000 - mae: 100654.3516 - mse: 41557086208.0000
Epoch 420/500
352/352 [==============================] - 1s 3ms/step - loss: 41513373696.0000 - mae: 100465.5703 - mse: 41513373696.0000
Epoch 421/500
352/352 [==============================] - 1s 3ms/step - loss: 41523384320.0000 - mae: 100553.0938 - mse: 41523384320.0000
Epoch 422/500
352/352 [==============================] - 1s 3ms/step - loss: 41512923136.0000 - mae: 100608.9922 - mse: 41512923136.0000
Epoch 423/500
352/352 [==============================] 

352/352 [==============================] - 1s 2ms/step - loss: 41062785024.0000 - mae: 99809.9766 - mse: 41062785024.0000
Epoch 477/500
352/352 [==============================] - 1s 2ms/step - loss: 41032708096.0000 - mae: 99635.4531 - mse: 41032708096.0000
Epoch 478/500
352/352 [==============================] - 1s 2ms/step - loss: 41034113024.0000 - mae: 99825.6797 - mse: 41034113024.0000
Epoch 479/500
352/352 [==============================] - 1s 2ms/step - loss: 41024909312.0000 - mae: 99617.1953 - mse: 41024909312.0000
Epoch 480/500
352/352 [==============================] - 1s 2ms/step - loss: 41009864704.0000 - mae: 99699.3984 - mse: 41009864704.0000
Epoch 481/500
352/352 [==============================] - 1s 2ms/step - loss: 41005690880.0000 - mae: 99547.8906 - mse: 41005690880.0000
Epoch 482/500
352/352 [==============================] - 1s 2ms/step - loss: 40997093376.0000 - mae: 99684.1797 - mse: 40997093376.0000
Epoch 483/500
352/352 [==============================] - 1s 2m

In [30]:
# Evaluate the model using the test data
errors = nn.evaluate(X_test_scaled,y_test,verbose=2)
errors
print(f"MSE: {errors [0]}, MAE: {errors[1]}")

118/118 - 0s - loss: 38567620608.0000 - mae: 100900.9688 - mse: 38567620608.0000 - 221ms/epoch - 2ms/step
MSE: 38567620608.0, MAE: 100900.96875


In [17]:
# Evaluate the model using the test data
errors = nn.evaluate(X_test_scaled,y_test,verbose=2)
errors
print(f"MSE: {errors [0]}, MAE: {errors[1]}")

118/118 - 0s - loss: 38586101760.0000 - mae: 100804.6328 - mse: 38586101760.0000 - 128ms/epoch - 1ms/step
MSE: 38586101760.0, MAE: 100804.6328125


In [18]:
nn.predict([X_train_scaled.tolist()[1]])


1/1 [==============================] - 0s 78ms/step


array([[546181.75]], dtype=float32)

In [19]:
# Export our model to HDF5 file
nn.save("../Resources/nn/neural_network_simplified.h5")

In [20]:
# Export standardScaler from sklearn to file
from joblib import dump, load
dump(scaler, '../Resources/nn/std_scaler_simplified.bin', compress=True)

['../Resources/nn/std_scaler_simplified.bin']

## Predict prices for original table

In [21]:
X_scaled=scaler.transform(X)
predict_price_nn=nn.predict(X_scaled).flatten()
predict_price_nn

469/469 [==============================] - 1s 1ms/step


array([ 492215.7 ,  285340.66,  228567.08, ...,  647062.9 , 1645918.5 ,
        371168.66], dtype=float32)

In [22]:
hpricing_extra_nn=pd.DataFrame()
hpricing_extra_nn["predict_price_nn"]=predict_price_nn
hpricing_extra_nn["latestPrice"]=hpricing_df_og["latestPrice"]
hpricing_extra_nn["difference_nn"]=hpricing_extra_nn["latestPrice"]-hpricing_extra_nn["predict_price_nn"]
hpricing_extra_nn["percent_change_nn"]=round(hpricing_extra_nn["difference_nn"]*100/hpricing_extra_nn["latestPrice"],2)
hpricing_extra_nn.head(10)

,predict_price_nn,latestPrice,difference_nn,percent_change_nn
0,492215.687500,305000,-187215.687500,-61.38
1,285340.656250,295000,9659.343750,3.27
2,228567.078125,256125,27557.921875,10.76
3,257835.859375,240000,-17835.859375,-7.43
4,411257.593750,239900,-171357.593750,-71.43
5,257438.421875,309045,51606.578125,16.70
6,461106.500000,315000,-146106.500000,-46.38
7,243038.437500,219900,-23138.437500,-10.52
8,345626.468750,225000,-120626.468750,-53.61
9,230413.953125,194800,-35613.953125,-18.28


In [23]:
hpricing_extra_nn.drop(columns=["latestPrice"],inplace=True)
hpricing_extra_nn.head(5)

,predict_price_nn,difference_nn,percent_change_nn
0,492215.687500,-187215.687500,-61.38
1,285340.656250,9659.343750,3.27
2,228567.078125,27557.921875,10.76
3,257835.859375,-17835.859375,-7.43
4,411257.593750,-171357.593750,-71.43


In [24]:
hpricing_extra_nn.to_csv("../Resources/austin_extra_nn_simplified.csv",index=False)

In [25]:
hpricing_extra_nn.mean()

predict_price_nn     510453.612807
difference_nn         -2007.062663
percent_change_nn        -7.658365
dtype: float64

In [26]:
hpricing_extra_nn.mean()

predict_price_nn     510453.612807
difference_nn         -2007.062663
percent_change_nn        -7.658365
dtype: float64